How many rented units were affordable to households at different incomes in 1980, 1990, 2000, 2010 and 2023?

In [1]:
import pandas as pd
import numpy as np

IPUMS v3 includes household income, built year 1 and 2, bedrooms, rent, gross rent for 1980, 1990, 2000, 2005 (ACS), 2010 (ACS), 2014 (ACS), 2023 (ACS) 

In [2]:
# load ipums data
df = pd.read_csv('source/ipums_v3.csv', low_memory=False)
df.head(3)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,CPI99,CITY,STRATA,GQ,OWNERSHP,OWNERSHPD,RENT,RENTGRS,HHINCOME,BUILTYR,BUILTYR2,BEDROOMS
0,1980,198002,550665,NaN,100,1980005506652,2.295,1190,21,1,2,22,165,165,16005,7.0,NaN,2
1,1980,198002,550666,NaN,100,1980005506662,2.295,1190,21,1,2,22,212,235,18005,5.0,NaN,2
2,1980,198002,550667,NaN,100,1980005506672,2.295,1190,35,1,2,22,325,415,12535,6.0,NaN,5


In [3]:
# cleanup

# convert year to string
df['YEAR'] = df['YEAR'].astype(str)

# remove duplicate household serials
df.drop_duplicates(subset=['SERIAL'], keep='first', inplace=True)

In [4]:
# filter out NA income
df['adj_hhincome'] = np.where(df['HHINCOME']!=9999999,df['HHINCOME']/12, np.nan)

In [5]:
# adjust rents and incomes for inflation with these constants: https://usa.ipums.org/usa/cpi99.shtml
# adjust to 1999 base year
# for acs years, most recent year is the data dollar year

df['adj_rent'] = (df['RENTGRS'] * df['CPI99']) # adjust to 1999
df['adj_hhincome'] = (df['HHINCOME'] * df['CPI99']) # adjust to 1999

# adjust from 1999 base year to 2024 with BLS CPI-U
# https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-
df['adj_rent'] = df['adj_rent'] * (314.4/166.6) # adjust to 2024
df['adj_hhincome'] = df['adj_hhincome'] * (314.4/166.6) # adjust to 2024

In [6]:
# check total households looks right?
df.groupby('YEAR')['HHWT'].sum()

YEAR
1980    1140300
1990    1007025
2000    1124221
2005     868164
2010    1055521
2014     880277
2023    1170677
Name: HHWT, dtype: int64

In [7]:
# create pivot the number of households at each rent
pivot = pd.pivot_table(df[df['adj_rent'] != 0],
              index='adj_rent',
              columns='YEAR',
              values='HHWT',
              aggfunc='sum')

pivot.to_csv('processed/rent_pivot_by_year.csv')

pivot = pivot.reset_index()
pivot

YEAR,adj_rent,1980,1990,2000,2005,2010,2014,2023
0,5.314228,NaN,NaN,NaN,NaN,NaN,209.0,NaN
1,18.871549,NaN,NaN,108.0,NaN,NaN,NaN,NaN
2,20.758703,NaN,NaN,123.0,NaN,NaN,NaN,NaN
3,26.420168,NaN,NaN,124.0,NaN,NaN,NaN,NaN
4,28.835726,NaN,NaN,NaN,NaN,150.0,NaN,NaN
...,...,...,...,...,...,...,...,...
4080,5644.472643,NaN,NaN,NaN,NaN,NaN,NaN,99.0
4081,5883.960144,NaN,NaN,NaN,NaN,NaN,NaN,83.0
4082,5966.542041,NaN,NaN,NaN,NaN,NaN,NaN,79.0
4083,5987.187515,NaN,NaN,NaN,NaN,NaN,NaN,95.0


In [8]:
# load 2024-adjusted AMIs for each year
# source: NHGIS
# https://docs.google.com/spreadsheets/d/1C3ToVnNv3JRd01gKtjIfcY_IeDyKyOS1pvqflSLynAU/edit?usp=sharing

ami = {'1980': 66262, # decennial
       '1990': 66686, # decennial
       '2000': 72891, # decennial
       '2010': 64546, # acs 1-year,
       '2014': 64732, #acs 1-year
       '2023': 76845} # acs 1-year

'''
OLD
ami = {'1980': 64197, # decennial
       '1990': 64623, # decennial
       '2000': 70612, # decennial
       '2010': 62539, # acs 1-year,
       '2014': 62622, #acs 1-year
       '2023': 74474} # acs 1-year
'''

"\nOLD\nami = {'1980': 64197, # decennial\n       '1990': 64623, # decennial\n       '2000': 70612, # decennial\n       '2010': 62539, # acs 1-year,\n       '2014': 62622, #acs 1-year\n       '2023': 74474} # acs 1-year\n"

In [9]:
# create a df that has the % of affordable units at each 10% AMI interval for all years

ami_pcts = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]

dfdict = {'year': [],
         'ami': [],
         'ami_pct': [],
         'ami_pct_value': [],
         'ami_pct_monthly': [],
         'ami_pct_aff_threshold': [],
         'aff_units': [],
         'pct_aff_units': []
         }

for year in ami.keys():
    for pct in ami_pcts:
        dfdict['year'].append(year)
        dfdict['ami'].append(ami[year])
        dfdict['ami_pct'].append(pct)
        
        ami_pct_value = ami[year] * pct # %ami
        dfdict['ami_pct_value'].append(ami_pct_value)
        
        ami_pct_monthly = ami_pct_value / 12 # %ami per month
        dfdict['ami_pct_monthly'].append(ami_pct_monthly)
        
        ami_pct_aff_threshold = ami_pct_monthly * 0.3 # 30% of monthly income
        dfdict['ami_pct_aff_threshold'].append(ami_pct_aff_threshold)

        aff_units = pivot.loc[pivot['adj_rent'] < ami_pct_aff_threshold, year].sum() # calc num of rentals less than threshold
        dfdict['aff_units'].append(aff_units)

        pct_aff_units = aff_units / pivot[year].sum()
        dfdict['pct_aff_units'].append(pct_aff_units)

In [10]:
output = pd.DataFrame(dfdict)

In [11]:
output.head(20)

,year,ami,ami_pct,ami_pct_value,ami_pct_monthly,ami_pct_aff_threshold,aff_units,pct_aff_units
0,1980,66262,0.1,6626.2,552.183333,165.655,3100.0,0.004885
1,1980,66262,0.2,13252.4,1104.366667,331.310,29000.0,0.045698
2,1980,66262,0.3,19878.6,1656.550000,496.965,50500.0,0.079578
3,1980,66262,0.4,26504.8,2208.733333,662.620,92300.0,0.145446
4,1980,66262,0.5,33131.0,2760.916667,828.275,186900.0,0.294516
5,1980,66262,0.6,39757.2,3313.100000,993.930,321500.0,0.506618
6,1980,66262,0.7,46383.4,3865.283333,1159.585,438100.0,0.690356
7,1980,66262,0.8,53009.6,4417.466667,1325.240,513600.0,0.809329
8,1980,66262,0.9,59635.8,4969.650000,1490.895,559200.0,0.881185
9,1980,66262,1.0,66262.0,5521.833333,1656.550,584100.0,0.920422


In [12]:
output.tail(20)

,year,ami,ami_pct,ami_pct_value,ami_pct_monthly,ami_pct_aff_threshold,aff_units,pct_aff_units
100,2023,76845,0.1,7684.5,640.375,192.1125,2257.0,0.003794
101,2023,76845,0.2,15369.0,1280.750,384.2250,23052.0,0.038749
102,2023,76845,0.3,23053.5,1921.125,576.3375,39311.0,0.066079
103,2023,76845,0.4,30738.0,2561.500,768.4500,61642.0,0.103616
104,2023,76845,0.5,38422.5,3201.875,960.5625,117212.0,0.197025
105,2023,76845,0.6,46107.0,3842.250,1152.6750,191340.0,0.321628
106,2023,76845,0.7,53791.5,4482.625,1344.7875,255044.0,0.428710
107,2023,76845,0.8,61476.0,5123.000,1536.9000,328101.0,0.551514
108,2023,76845,0.9,69160.5,5763.375,1729.0125,379355.0,0.637668
109,2023,76845,1.0,76845.0,6403.750,1921.1250,425662.0,0.715507


A WBEZ analysis finds that a household making the city’s median income in 1980 could afford more than 90 percent of apartments in the city. That means they would spend no more than roughly a third of their monthly income on rent and utilities. By 2023, just about 70 percent of apartments were affordable. 

In [13]:
# export as csv
output.to_csv('output/aff_units_by_ami_threshold.csv')